In [1]:
import numpy as np
import torch
from captum.attr import ShapleyValueSampling
from tqdm import trange

from load_data import load_data
from train_models import *
from segmentation import *
from utils import *
import os

In [2]:
# to utils.py

def change_points_to_lengths(change_points, array_length):
	# change points is 1D iterable of idxs
	# assumes that each change point is the start of a new segment, aka change_points = start points
	start_points = np.array(change_points)
	end_points = np.append(change_points[1:], [array_length])
	print(start_points, end_points)
	lengths = end_points - start_points
	return lengths

def lengths_to_weights(lengths):
	# lengths is 1D iterable of positive ints
	start_idx = 0
	end_idx = 0
	segment_weights = 1 / lengths
	weights = np.ones(lengths.sum())
	for segment_weight, length in zip(segment_weights, lengths):
		end_idx += length
		weights[start_idx: end_idx] = segment_weight
		start_idx = end_idx
	return weights



In [3]:
# device for torch
from torch.cuda import is_available as is_GPU_available
device = "cuda" if is_GPU_available() else "cpu"

# dictionary mapping predictors to torch vs other, necessary for Captum 
predictors = {
	'torch' : ['resNet'],
	'scikit' : ['miniRocket','randomForest']
}

In [4]:
# load data
dataset_name = 'gunpoint'
predictor_name = 'resNet'

# I've returned also a Label encoder from load_data to have a mapping between dataset label
# which can be string while captum requires idx (integers)
X_train, X_test, y_train, y_test, enc = load_data(subset='all', dataset_name=dataset_name)

# train model
if predictor_name=='resNet':
	clf,preds = train_ResNet(X_train, y_train, X_test, y_test, dataset_name,device=device)
elif predictor_name=='miniRocket':
	clf,preds = train_miniRocket(X_train, y_train, X_test, y_test, dataset_name)

# create a dictionary to be dumped containing attribution and metadata
# initialize data structure meant to contain the segments
segments =  np.empty( (X_test.shape[0] , X_test.shape[1]), dtype=object) if X_test.shape[1] > 1  else (
	np.empty( X_test.shape[0] , dtype=object))

results = {
	'attributions' : {},
	'segments' : segments,
	'y_test_true' : y_test,
	'y_test_pred' : preds,
	'label_mapping' : enc,
}

training ResNet
Epoch 1: train loss:  0.671, 	 train accuracy  0.520 
          test loss:  0.588,  	 test accuracy  0.667
Epoch 11: train loss:  0.239, 	 train accuracy  0.980 
          test loss:  0.293,  	 test accuracy  0.927
Epoch 21: train loss:  0.130, 	 train accuracy  1.000 
          test loss:  0.151,  	 test accuracy  1.000
Epoch 31: train loss:  0.078, 	 train accuracy  1.000 
          test loss:  0.100,  	 test accuracy  1.000
Epoch 41: train loss:  0.069, 	 train accuracy  1.000 
          test loss:  0.085,  	 test accuracy  1.000
training early stopped! Final stats are:
Epoch 48: train loss:  0.035, 	 train accuracy  1.000 
          test loss:  0.070,  	 test accuracy  1.000
accuracy for resNet is  1.0


In [7]:
n_background = 50
background_types = ["average","zero","sampling"] # zero, constant, average, multisample

# TODO for each baseline so that I don't retrain a model each time 

with torch.no_grad():
    SHAP = ShapleyValueSampling(clf) if predictor_name in predictors['torch'] else ShapleyValueSampling(forward_classification)
    
    for i in trange ( X_test.shape[0] ) : #
    
        # get current sample and label
        ts, y = X_test[i] , torch.tensor( y_test[i:i+1] )
        
        # get segment and its tensor representation
        current_segments = get_claSP_segmentation(ts)[:X_test.shape[1]]
        results['segments'][i] = current_segments
        mask = get_feature_mask(current_segments,ts.shape[-1])
  
        ts = torch.tensor(ts).repeat(1,1,1)
    
        for background_type in background_types:
        
            results['attributions'][background_type] = np.empty( X_test.shape ,dtype=np.float32 )
        
            # background data
            if background_type=="zero":
                background_dataset = torch.zeros((1,) + X_train.shape[1:])
            elif background_type=="sampling":
                background_dataset = sample_background(X_train, n_background)
            elif background_type=="average":
                background_dataset = sample_background(X_train, n_background).mean(axis=0, keepdim=True)
        
            ts = ts.repeat(background_dataset.shape[0],1,1) if background_type=="sampling" else ts
        
            # data structure with room for each sample in the background dataset
            if predictor_name in predictors['scikit']:
                tmp = SHAP.attribute( ts, target=y , feature_mask=mask, baselines=background_dataset, additional_forward_args=clf)
            elif predictor_name in predictors['torch']:
                ts = ts.to(device); y = y.to(device)
                mask = mask.to(device) ; background_dataset =  background_dataset.to(device)
        
                tmp = SHAP.attribute( ts, target=y , feature_mask=mask, baselines=background_dataset)
                ########  only for random forest as every instance should be a 1D tensor    ########
                #current_attr[j:j+actual_size] = tmp.reshape(actual_size,X_test.shape[1],X_test.shape[2])
                ###############################################################################
        
            # compute as final explanation mean of each explanation using a different baseline
            results['attributions'][background_type][i] = torch.mean(tmp, dim=0).cpu().numpy() if \
                background_type=="sampling" else tmp.cpu().numpy()

100%|██████████| 150/150 [02:10<00:00,  1.15it/s]


In [11]:
 # normalized weights
weights = np.array(list(map(lambda x: list(map(lambda y: lengths_to_weights(change_points_to_lengths(y, X_train.shape[-1])), x)), results["segments"])))
results["attributions"][background_type] *= weights

[  0   7  16  47  96 109 118] [  7  16  47  96 109 118 150]
[  0  35  45  68  78 101 132] [ 35  45  68  78 101 132 150]
[  0  23  47  69 106 119 131] [ 23  47  69 106 119 131 150]
[  0  27  61  96 111 119 135] [ 27  61  96 111 119 135 150]
[0] [150]
[  0   9  19  42  63 112 129] [  9  19  42  63 112 129 150]
[ 0 56 64 72 86 99] [ 56  64  72  86  99 150]
[  0 123] [123 150]
[  0  27  42  80 110 124 141] [ 27  42  80 110 124 141 150]
[  0  15  41  57  87 100 132] [ 15  41  57  87 100 132 150]
[  0  80  87  99 108 119 137] [ 80  87  99 108 119 137 150]
[ 0 10 39 46 63 88 97] [ 10  39  46  63  88  97 150]
[  0   9  18  39  77 114 139] [  9  18  39  77 114 139 150]
[  0  79  99 131 138] [ 79  99 131 138 150]
[  0  15  65  88 103 119 138] [ 15  65  88 103 119 138 150]
[ 0 12 83] [ 12  83 150]
[  0  34  69 120] [ 34  69 120 150]
[ 0  8 19 34 42 77 98] [  8  19  34  42  77  98 150]
[0] [150]
[  0  33  40  48  65  77 106] [ 33  40  48  65  77 106 150]
[  0  26 129] [ 26 129 150]
[  0  16  38  7

In [9]:
# dump result to disk
file_name = "_".join ( (predictor_name,dataset_name) )+".npy"
file_path = os.path.join("attributions", file_name)
np.save( file_path, results )

In [10]:
results['attributions'][background_type].sum(axis=(1,2))

array([-9.56390083e-01,  1.16674888e+00, -9.35321808e-01, -2.09154630e+00,
        0.00000000e+00, -2.31724024e+00,  3.04134130e+00,  2.32914448e-01,
       -1.60881102e+00, -4.77749467e-01,  2.40988350e+00,  1.65474248e+00,
       -3.25472403e+00, -1.07835209e+00, -1.11364555e+00, -6.46999717e-01,
       -9.66670513e-02,  3.48845720e-01,  0.00000000e+00,  2.36834955e+00,
       -1.00147939e+00,  3.35213482e-01, -1.20195460e+00,  5.22527516e-01,
        9.73926544e-01, -2.35491610e+00, -1.34536564e+00,  4.10208464e-01,
       -2.00159597e+00, -9.68097568e-01,  1.86761975e+00, -1.96810460e+00,
       -1.70643485e+00, -1.25338584e-01, -2.20549047e-01, -1.71060979e+00,
       -1.42822599e+00, -1.99016660e-01, -8.47679138e-01, -2.16994834e+00,
       -8.49690735e-01, -1.80075645e+00, -9.20104146e-01, -1.11758709e-08,
        0.00000000e+00, -1.27716613e+00, -4.94556427e-01,  6.41708255e-01,
       -2.58888578e+00, -1.17992020e+00,  8.71414661e-01, -1.26081228e-01,
        2.19527149e+00,  